In [5]:
!pip install pyro-ppl==1.3.0

     |████████████████████████████████| 495 kB 658 kB/s eta 0:00:01
  Attempting uninstall: pyro-ppl
    Found existing installation: pyro-ppl 1.3.1
    Uninstalling pyro-ppl-1.3.1:
      Successfully uninstalled pyro-ppl-1.3.1


### Contrasting Pyro HMC Vs. statsmodel results
**For a regression model trained on country topographical rugged index and GDP data.**

#### A. Pyro model training & HMC sampling

In [1]:
import os

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.distributions import constraints

import pyro
import pyro.distributions as dist
import pyro.optim as optim

DATA_URL = "https://d2hg8soec8ck9v.cloudfront.net/datasets/rugged_data.csv"
rugged_data = pd.read_csv(DATA_URL, encoding="ISO-8859-1")

* **Define pyro model and guide:**
    * Priors are deined in model function for intercept `b`, feature 1 `b_a`, feature 2 `b_r`, feature 3 `b_ar`
    * mean is the expresion `f(feat1, feat2, feat3, b)`

In [2]:
def model(is_cont_africa, ruggedness, log_gdp):
    b = pyro.sample("bias", dist.Normal(0., 10.))
    b_a = pyro.sample("feat_1", dist.Normal(0., 1.))
    b_r = pyro.sample("feat_2", dist.Normal(0., 1.))
    b_ar = pyro.sample("feat_3", dist.Normal(0., 1.))
    sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
    mean = b + b_a * is_cont_africa + b_r * ruggedness + b_ar * is_cont_africa * ruggedness
    with pyro.plate("data", len(ruggedness)):
        pyro.sample("obs", dist.Normal(mean, sigma), obs=log_gdp)

In [ ]:
def guide(is_cont_africa, ruggedness, log_gdp):
    b_loc = pyro.param('bias_loc', torch.tensor(0.))
    b_scale = pyro.param('bias_scale', torch.tensor(1.),
                         constraint=constraints.positive)
    sigma_loc = pyro.param('sigma_loc', torch.tensor(1.),
                             constraint=constraints.positive)
    weights_loc = pyro.param('weights_loc', torch.randn(3))
    weights_scale = pyro.param('weights_scale', torch.ones(3),
                               constraint=constraints.positive)
    b = pyro.sample("bias", dist.Normal(b_loc, b_scale))
    b_a = pyro.sample("feat_1", dist.Normal(weights_loc[0], weights_scale[0]))
    b_r = pyro.sample("feat_2", dist.Normal(weights_loc[1], weights_scale[1]))
    b_ar = pyro.sample("feat_3", dist.Normal(weights_loc[2], weights_scale[2]))
    sigma = pyro.sample("sigma", dist.Normal(sigma_loc, torch.tensor(0.05)))
    mean = a + b_a * is_cont_africa + b_r * ruggedness + b_ar * is_cont_africa * ruggedness

* **Loading data**

In [3]:
# Prepare training data
df = rugged_data[["cont_africa", "rugged", "rgdppc_2000"]]
df = df[np.isfinite(df.rgdppc_2000)]
df["rgdppc_2000"] = np.log(df["rgdppc_2000"])
train = torch.tensor(df.values, dtype=torch.float)

is_cont_africa, ruggedness, log_gdp = train[:, 0], train[:, 1], train[:, 2]

* **HMC Sampling**

In [4]:
from pyro.infer import MCMC, NUTS

nuts_kernel = NUTS(model)

mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200)
mcmc.run(is_cont_africa, ruggedness, log_gdp)

hmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}


Sample: 100%|██████████| 1200/1200 [00:32, 36.52it/s, step size=3.66e-01, acc. prob=0.945]


* **Summary of hmc samples**

In [5]:
# Utility function to print latent sites' quantile information.
def summary(samples):
    site_stats = {}
    for site_name, values in samples.items():
        marginal_site = pd.DataFrame(values)
        describe = marginal_site.describe(percentiles=[.05, 0.25, 0.5, 0.75, 0.95]).transpose()
        site_stats[site_name] = describe[["mean", "std", "5%", "25%", "50%", "75%", "95%"]]
    return site_stats

hmc_df= pd.DataFrame()
for key in summary(hmc_samples).keys():
    temp = summary(hmc_samples)[key]
    temp.index= [key]
    hmc_df= pd.concat([hmc_df, temp])

hmc_df

,mean,std,5%,25%,50%,75%,95%
bias,9.183471,0.134134,8.955453,9.091243,9.186186,9.268926,9.415138
feat_1,-1.846470,0.220783,-2.209519,-1.986917,-1.844962,-1.690313,-1.492916
feat_2,-0.185189,0.074806,-0.316412,-0.234034,-0.181285,-0.133148,-0.063197
feat_3,0.351885,0.127043,0.143173,0.266875,0.354971,0.439539,0.552472
sigma,0.948347,0.049657,0.873508,0.913886,0.946328,0.979219,1.031998


#### B. Statsmodel training

**Defining feature_3 for statsmodel**
* For statsmodel training, there are 3 features (`cont_africa`,`rugged`,`cont_africa_x_rugged`), 1 bias and 1 target label `rgdppc_2000`(GDP_per_capita)

In [6]:
df["cont_africa_x_rugged"] = df["cont_africa"] * df["rugged"]
df.head(3)

,cont_africa,rugged,rgdppc_2000,cont_africa_x_rugged
2,1,0.858,7.492609,0.858
4,0,3.427,8.216929,0.000
7,0,0.769,9.933263,0.000


In [7]:
import statsmodels.api as sm
feats, targets= df[['cont_africa', 'rugged', 'cont_africa_x_rugged']].values, df['rgdppc_2000'].values
feats= sm.add_constant(feats, prepend=False)

In [8]:
model_stm = sm.OLS(targets, feats, hasconst=True)
result= model_stm.fit(disp=0)
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.345
Method:                 Least Squares   F-statistic:                     30.71
Date:                Mon, 20 Apr 2020   Prob (F-statistic):           7.60e-16
Time:                        17:24:13   Log-Likelihood:                -229.37
No. Observations:                 170   AIC:                             466.7
Df Residuals:                     166   BIC:                             479.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -1.9480      0.227     -8.572      0.0

* **Summary of statsmodel result**

In [9]:
stats_summary= result.summary()
statsm_df = pd.DataFrame(stats_summary.tables[1].data[1:], columns=stats_summary.tables[1].data[0])
statsm_df

,,coef,std err,t,P>|t|,[0.025,0.975]
0,x1,-1.9480,0.227,-8.572,0.000,-2.397,-1.499
1,x2,-0.2029,0.077,-2.621,0.010,-0.356,-0.050
2,x3,0.3934,0.132,2.989,0.003,0.134,0.653
3,const,9.2232,0.140,66.044,0.000,8.948,9.499


#### Contrasting resulst from HMC sampling and Statsmodel training

In [11]:
hmc_statsmodel_keys= {'bias':3, 'feat_1':0,'feat_2':1,'feat_3':2}#{0:X1,1:X2,2:X3, 3:const}

stats_hmc_di= dict()
for key, val in hmc_statsmodel_keys.items():
    print("For feature {}:".format(key))#site
    print('. . .val from HMC obtained samples: {}',hmc_df['mean'][key])
    print('. . .val from statsmodel training: {}',statsm_df['coef'][val], "\n")
    stats_hmc_di['param_val_{}'.format(key)]= [hmc_df['mean'][key], statsm_df['coef'][val]]
    stats_hmc_di['std_error_{}'.format(key)]= [hmc_df['std'][key], statsm_df['std err'][val]]

For feature bias:
. . .val from HMC obtained samples: {} 9.183470726013184
. . .val from statsmodel training: {}     9.2232 

For feature feat_1:
. . .val from HMC obtained samples: {} -1.8464702367782593
. . .val from statsmodel training: {}    -1.9480 

For feature feat_2:
. . .val from HMC obtained samples: {} -0.1851891428232193
. . .val from statsmodel training: {}    -0.2029 

For feature feat_3:
. . .val from HMC obtained samples: {} 0.35188522934913635
. . .val from statsmodel training: {}     0.3934 



In [12]:
pd.DataFrame(stats_hmc_di, index= ['hmc_samples','statsmodel_results'])

,param_val_bias,std_error_bias,param_val_feat_1,std_error_feat_1,param_val_feat_2,std_error_feat_2,param_val_feat_3,std_error_feat_3
hmc_samples,9.18347,0.134134,-1.84647,0.220783,-0.185189,0.0748065,0.351885,0.127043
statsmodel_results,9.2232,0.140,-1.9480,0.227,-0.2029,0.077,0.3934,0.132


#### HMC sampling on explicit pytorch model

In [31]:
from pyro.nn import PyroSample
from torch import nn
from pyro.nn import PyroModule

class regression_model(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([out_features]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean

model2 = regression_model(3, 1)

In [35]:
data = torch.tensor(df[["cont_africa", "rugged", "cont_africa_x_rugged", "rgdppc_2000"]].values,
                        dtype=torch.float)
x_data, y_data = data[:, :-1], data[:, -1]

In [36]:
nuts_kernel = NUTS(model2)

mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=200)
mcmc.run(x_data, y_data)#is_cont_africa, ruggedness, log_gdp)
hmc_samples2 = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}

Sample: 100%|██████████| 1200/1200 [00:23, 50.12it/s, step size=4.40e-01, acc. prob=0.914]


In [77]:
hmc_samples_di = {'feat_1':hmc_samples2['linear.weight'].reshape(-1,3)[:,0],'feat_2':hmc_samples2['linear.weight'].reshape(-1,3)[:,1], 'feat_3':hmc_samples2['linear.weight'].reshape(-1,3)[:,2], 'bias':hmc_samples2['linear.bias'].reshape(-1)}#hmc_samples2['linear.weight'].reshape(-1,3)
hmc_df2= pd.DataFrame()
for key in summary(hmc_samples_di).keys():
    temp = summary(hmc_samples_di)[key]
    temp.index= [key]
    hmc_df2= pd.concat([hmc_df2, temp])

hmc_df2

,mean,std,5%,25%,50%,75%,95%
feat_1,-1.838981,0.223160,-2.207797,-1.997128,-1.839201,-1.685130,-1.478482
feat_2,-0.180838,0.078172,-0.304442,-0.234397,-0.184867,-0.125720,-0.056102
feat_3,0.343805,0.133534,0.115104,0.252733,0.347964,0.435917,0.559253
bias,9.178177,0.140712,8.943239,9.081530,9.178934,9.274403,9.404644


In [78]:
hmc_statsmodel_keys= {'bias':3, 'feat_1':0,'feat_2':1,'feat_3':2}#{0:X1,1:X2,2:X3, 3:const}

stats_hmc_di_2= dict()
for key, val in hmc_statsmodel_keys.items():
    print("For feature {}:".format(key))#site
    print('. . .val from HMC obtained samples: {}',hmc_df2['mean'][key])
    print('. . .val from statsmodel training: {}',statsm_df['coef'][val], "\n")
    stats_hmc_di_2['param_val_{}'.format(key)]= [hmc_df2['mean'][key], statsm_df['coef'][val]]
    stats_hmc_di_2['std_error_{}'.format(key)]= [hmc_df2['std'][key], statsm_df['std err'][val]]

For feature bias:
. . .val from HMC obtained samples: {} 9.178176879882812
. . .val from statsmodel training: {}     9.2232 

For feature feat_1:
. . .val from HMC obtained samples: {} -1.8389813899993896
. . .val from statsmodel training: {}    -1.9480 

For feature feat_2:
. . .val from HMC obtained samples: {} -0.18083825707435608
. . .val from statsmodel training: {}    -0.2029 

For feature feat_3:
. . .val from HMC obtained samples: {} 0.34380537271499634
. . .val from statsmodel training: {}     0.3934 



In [79]:
pd.DataFrame(stats_hmc_di_2, index= ['hmc_samples','statsmodel_results'])

,param_val_bias,std_error_bias,param_val_feat_1,std_error_feat_1,param_val_feat_2,std_error_feat_2,param_val_feat_3,std_error_feat_3
hmc_samples,9.17818,0.140712,-1.83898,0.22316,-0.180838,0.0781725,0.343805,0.133534
statsmodel_results,9.2232,0.140,-1.9480,0.227,-0.2029,0.077,0.3934,0.132
